## Descriptors
### Дескрипторы
#### Кратко:
- `__get__` - нужно что-то получить (обращение на чтение)
- `__set__` - нужно что-то сохранить (обращение на установку значения)
- `__del__` - нужно что-то удалить (удаление свойства)

Любое свойство — дескриптор.

In [1]:
class User:
    name = "Sam"

In [2]:
User.name

'Sam'

In [3]:
type.__getattribute__(User, "name")

'Sam'

In [4]:
type.__getattribute__(User, "__dict__")

mappingproxy({'__module__': '__main__',
              'name': 'Sam',
              '__dict__': <attribute '__dict__' of 'User' objects>,
              '__weakref__': <attribute '__weakref__' of 'User' objects>,
              '__doc__': None})

In [5]:
User.__dict__

mappingproxy({'__module__': '__main__',
              'name': 'Sam',
              '__dict__': <attribute '__dict__' of 'User' objects>,
              '__weakref__': <attribute '__weakref__' of 'User' objects>,
              '__doc__': None})

In [6]:
user = User()
print(user)
print(user.__dict__)

{}


In [7]:
user.name

'Sam'

In [8]:
object.__getattribute__(user, "name")

'Sam'

In [9]:
vars(user)

{}

In [10]:
user.name = "John"
print(User.name)
print(user.name)

Sam
John


In [11]:
vars(user)

{'name': 'John'}

In [12]:
object.__getattribute__(user, "name")

'John'

In [13]:
type.__getattribute__(User, "name")

'Sam'

In [15]:
class User:
    def __init__(self, username):
        self.username = username

    @property
    def email(self):
        return f"{self.username}@example.com".lower()

In [16]:
user = User("John")
print(user.username)
print(user.email)

John
john@example.com


In [17]:
class EmailDescriptor:
    def __get__(self, instance, owner):
        return f"{instance.username}@example.com".lower()

In [18]:
class User:
    def __init__(self, username):
        self.username = username

    email = EmailDescriptor()

In [19]:
user = User("John")
print(user.username)
print(user.email)

John
john@example.com


In [20]:
vars(User)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.User.__init__(self, username)>,
              'email': <__main__.EmailDescriptor at 0x1072a23d0>,
              '__dict__': <attribute '__dict__' of 'User' objects>,
              '__weakref__': <attribute '__weakref__' of 'User' objects>,
              '__doc__': None})

In [21]:
vars(user)

{'username': 'John'}

In [22]:
class EmailDescriptor:
    def __init__(self, domain):
        self.domain = domain

    def __get__(self, instance, owner):
        return f"{instance.username}@{self.domain}".lower()

In [23]:
class User:
    def __init__(self, username):
        self.username = username

    email = EmailDescriptor(domain="example.org")

In [24]:
user = User("John")
print(user.username)
print(user.email)

John
john@example.org


In [25]:
User.email

AttributeError: 'NoneType' object has no attribute 'username'

In [26]:
vars(User)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.User.__init__(self, username)>,
              'email': <__main__.EmailDescriptor at 0x1072cd550>,
              '__dict__': <attribute '__dict__' of 'User' objects>,
              '__weakref__': <attribute '__weakref__' of 'User' objects>,
              '__doc__': None})

In [27]:
vars(User)["email"]

In [28]:
vars(User)["email"].domain = "abc.org"

In [29]:
vars(User)["email"].domain

'abc.org'

In [30]:
print(user.username)
print(user.email)

John
john@abc.org


In [41]:
from abc import ABC, abstractmethod

class VerboseValidator(ABC):

    def __set_name__(self, owner, name):
        self.attr_name = "_" + name

    def __get__(self, instance, owner):
        print("get attr", self.attr_name)
        return getattr(instance, self.attr_name)

    def __set__(self, instance, value):
        print("validate", repr(value), "before setting to", self.attr_name)
        value = self.validate(value)
        print("set", repr(value), "to attr", self.attr_name)
        setattr(instance, self.attr_name, value)
    
    @abstractmethod
    def validate(self, value):
        pass


class LowerString(VerboseValidator):
    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f"Value {value} has to be type {str}, not {type(value)}")
        return value.lower()


class Number(VerboseValidator):
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value

    def validate(self, value):
        if not isinstance(value, int):
            raise TypeError(f"Value {value} has to be type {int}, not {type(value)}")

        value = int(value)
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"Value {value} is less than {self.min_value}")

        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"Value {value} is greater than {self.max_value}")

        return value


class Node:
    name = LowerString()
    value = Number(min_value=1, max_value=10)

    def __init__(self, name, value):
        self.name = name
        self.value = value

In [42]:
vars(Node)

mappingproxy({'__module__': '__main__',
              'name': <__main__.LowerString at 0x1073e5210>,
              'value': <__main__.Number at 0x1073e4590>,
              '__init__': <function __main__.Node.__init__(self, name, value)>,
              '__dict__': <attribute '__dict__' of 'Node' objects>,
              '__weakref__': <attribute '__weakref__' of 'Node' objects>,
              '__doc__': None})

In [43]:
Node.name

get attr _name


AttributeError: 'NoneType' object has no attribute '_name'

In [44]:
node = Node("Left", 7)
print(node.name)

validate 'Left' before setting to _name
set 'left' to attr _name
validate 7 before setting to _value
set 7 to attr _value
get attr _name
left


In [45]:
node._name

'left'

In [46]:
print(node.value)

get attr _value
7


In [47]:
vars(node)

{'_name': 'left', '_value': 7}

In [48]:
node.name = "Right"

validate 'Right' before setting to _name
set 'right' to attr _name


In [49]:
node.name

get attr _name


'right'

In [50]:
node.value = 11

validate 11 before setting to _value


ValueError: Value 11 is greater than 10

In [51]:
node.value

get attr _value


7

In [52]:
node.value = 5

validate 5 before setting to _value
set 5 to attr _value


In [53]:
node.value

get attr _value


5

In [54]:
Node.__getattribute__(node, "name")

get attr _name


'right'

In [55]:
node.__getattribute__("name")

get attr _name


'right'

In [56]:
node.__setattr__("name", "BoTtOm")

validate 'BoTtOm' before setting to _name
set 'bottom' to attr _name


In [57]:
node.__getattribute__("name")

get attr _name


'bottom'

In [58]:
node.name

get attr _name


'bottom'